In [1]:
import ipynb.fs.defs.functions_v2 as fct
import pandas as pd
import numpy as np
import pickle
import random
from scipy.stats import pearsonr, bernoulli

# Sim - refit

In [2]:
Nsim = 1

In [3]:
all_users_folder = 'data/all_users/'

# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

In [4]:
mod_info = {}
mod_info['name'] = 'mod000' 
mod_info['value_fct']= getattr(fct, 'rescorla_wagner_noV0_1trial')
mod_info['dec_fct']= getattr(fct, 'my_softmax_nobeta_1trial')
mod_info['param_names']= ['alpha']

value_fct = mod_info['value_fct']
dec_fct = mod_info['dec_fct']

In [5]:
# Parameter bounds
param_lower_bounds = [0]
param_upper_bounds = [1]

## Functions

In [6]:
def rescorla_wagner_noV0_1trial(vt_m_1, isHit, fb, alpha):    

    pe = np.nan

    # if hit, recieves fb
    if isHit == 1:
        # Compute prediction error
        pe = fb - vt_m_1
        # Compute new vt and fill in 
        vt = vt_m_1 + alpha * pe

    # if no hit, no fb
    elif isHit == 0:
        # vt does not change 
        vt = vt_m_1

    return vt, pe

In [7]:
def my_softmax_nobeta_1trial(vt):

    x = vt
    
    p_hit =  np.exp(x)/(np.exp(x)+1)
    
    #isHit = bernoulli.rvs(p_hit, size=1)
    if p_hit >= 0.5:
        isHit = 1
    else:
        isHit = 0

    return int(isHit)

In [8]:
def simulate_behav(fbs_all_cues, trialNo_all_cues, param_values, param_names):
    
    v0 = 0
        
    # Value function 
    # Free parameters
    alpha = param_values[param_names.index('alpha')]
    
    Ntrials = sum(len(lst) for lst in fbs_all_cues.values())

    # Initialise empty dictionary
    vt_all_cues = dict.fromkeys(fbs_all_cues.keys())
    pe_all_cues = dict.fromkeys(fbs_all_cues.keys())
    isHit_all_cues = dict.fromkeys(vt_all_cues.keys())
    
    # Iterate over cues
    for cue, fbs in zip(fbs_all_cues.keys(), fbs_all_cues.values()):

        # Initialise vectors
        vts = np.empty(len(fbs))
        hits = np.empty(len(fbs))
        pes = np.empty(len(fbs))
        
        # Fill in
        vts.fill(np.nan)
        hits.fill(np.nan)
        pes.fill(np.nan)

        # Fill in prior
        vts[0] = v0
        hits[0] = 0

        # Iterate to fill in vector
        for t in range(1,len(vts)):
            vt, pe = rescorla_wagner_noV0_1trial(vts[t-1], hits[t-1], fbs[t-1], alpha)
            p_hit = my_softmax_nobeta_1trial(vt)
            # Behaviour
            isHit = 1 if p_hit>=0.5 else 0
            vts[t] = vt
            pes[t] = pe
            hits[t] = isHit

        vt_all_cues[cue] = vts
        pe_all_cues[cue] = pes
        isHit_all_cues[cue] = hits
        
    return vt_all_cues, pe_all_cues, isHit_all_cues

# Main

In [9]:
# Sample parameters
params = np.array([])

for low_b, upp_b in zip(param_lower_bounds, param_upper_bounds):

    # Sample parameter from mean (resample if not within the bounds)
    param = np.nan
    param = np.random.uniform(low=low_b, high=upp_b, size=None)

    # Parameter value
    params = np.append(params, param)

# Store
mod_info['param_values'] = params
print(mod_info)

{'name': 'mod000', 'value_fct': <function rescorla_wagner_noV0_1trial at 0x7fafe9e34af0>, 'dec_fct': <function my_softmax_nobeta_1trial at 0x7fafea4585e0>, 'param_names': ['alpha'], 'param_values': array([0.73993124])}


In [10]:
# Create a model object
mod = fct.Model(mod_name = mod_info['name'],
             value_fct = mod_info['value_fct'], 
             dec_fct = mod_info['dec_fct'], 
             param_names = mod_info['param_names'])

In [11]:
# Extract random data set
random.shuffle(uniqueIDs)
ID = uniqueIDs[0]
user_folder = 'data/user_' + ID + '/'
df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
_, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)
print('Dataset from ID =',ID)

Dataset from ID = 335


In [13]:
# Simulate behaviour
vt_all_cues, pe_all_cues, isHit_all_cues = mod.simulate_behaviour(fbs_all_cues, trialNo_all_cues, mod_info['param_values'], mod_info['param_names'])

IndexError: list index out of range

In [ ]:
# Input data to model
mod.gen_param_values = mod_info['param_values']
mod.set_data('', fbs_all_cues, isHit_all_cues, trialNo_all_cues)

In [ ]:
# Fit model
mod.fit(param_lower_bounds, param_upper_bounds, n_iterations=8)

In [16]:
mod.gen_param_values

array([0.91474757])

In [17]:
mod.param_values

array([0.38196601])